# Download ORACC JSON Files
This script downloads open data from the Open Richly Annotated Cuneiform Corpus ([ORACC](http://oracc.org)) in `json` format. The JSON files are made available in a ZIP file. For a description of the various JSON files included in the ZIP see the [open data](http://oracc.org/doc/opendata) page on [ORACC](http://oracc.org). 

The code in this notebook is also available in the module `utils` in the directory `utils` and can be called as follows: 
```python
import os
import sys
util_dir = os.path.abspath('../utils') # When necessary, adapt the path to the utils directory.
sys.path.append(util_dir)
from utils import *
directories = ["jsonzip"]
make_dirs(directories)
projects = ["dcclt", "saao/saa01"] # or any list of ORACC projects
oracc_download(projects)
```

# 0. Import Packages

In [ ]:
import requests
from tqdm._tqdm_notebook import tqdm_notebook
import os
import errno

# 1. Create Download Directory
Create a directory called `jsonzip`. If the directory already exists, do nothing.

For the code, see [Stack Overflow](http://stackoverflow.com/questions/18973418/os-mkdirpath-returns-oserror-when-directory-does-not-exist).

In [ ]:
try:
    os.mkdir('jsonzip')
except OSError as exc:
    if exc.errno !=errno.EEXIST:
        raise
    pass

# 2.1 Input a List of Project Abbreviations
Enter one or more project abbreviations to download their JSON zip files. The project names are separated by commas. Note that the subprojects must be given explicitly, they are not automatically included in the main project. For instance: 
* saao/saa01, aemw/alalakh/idrimi, rimanum

In [ ]:
projects = input('Project(s): ').lower().strip() # lowercase user input and strip accidental spaces


# 2.2 Split the List of Projects
Split the list of projects and create a list of project names.

In [ ]:
p = projects.split(',')               # split at each comma and make a list called `p`
p = [x.strip() for x in p]        # strip spaces left and right of each entry in `p`

## Download the ZIP files
For each project from which files are to be processed download the entire project (all the json files) from `http://build-oracc.museum.upenn.edu/json/`. The file is called `PROJECT.zip` (for instance: `dcclt.zip`). For subprojects the file is called `PROJECT-SUBPROJECT.zip` (for instance `cams-gkab.zip`). 

For larger projects (such as [DCCLT](http://oracc.org/dcclt)) the `zip` file may be 25Mb or more. Downloading may take some time and it may be necessary to chunk the downloading process. The `iter_content()` function in the `requests` library takes care of that.

This code is also available as the function `oracc_download()` in the module `utils/utils.py`, which is used by some of the other scripts in Compass. The function `oracc_download()` takes a list of projects as its only argument (e.g. `['dcclt', 'hbtin', 'saao/saa01']` and will the same list of projects, omitting duplicates and invalid project names.

In [ ]:
CHUNK = 16 * 1024
for project in p:
    proj = project.replace('/', '-')
    url = "http://build-oracc.museum.upenn.edu/json/" + proj + ".zip"
    file = 'jsonzip/' + proj + '.zip'
    with requests.get(url, stream=True) as r:
        if r.status_code == 200:
            with open(file, 'wb') as f:
                for c in tqdm_notebook(r.iter_content(chunk_size=CHUNK), desc = project):
                    f.write(c)
        else:
            print(url + " does not exist.")